In [117]:
import pandas as pd

df = pd.read_excel("RnD.xlsx", index_col=False)
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,R&D expenditure (Mill. EUR),R&D personnel (HC),R&D in full-time equivalents (FTE)
0,2011.0,WHOLE COUNTRY,Total,7163.8,80817,54526.1
1,NaN,NaN,"ENTERPRISE SECTOR, TOTAL",5047.5,40949,31179.6
2,NaN,NaN,"GOVERNMENT SECTOR + PRIVATE NON-PROFIT SECTOR,...",684.4,9578,7499.7
3,NaN,NaN,"HIGHER EDUCATION SECTOR, TOTAL",1431.8,30290,15846.8
4,NaN,MA1 MAINLAND FINLAND,Total,7158.1,80720,54465.4
...,...,...,...,...,...,...
1087,NaN,NaN,"HIGHER EDUCATION SECTOR, TOTAL",40.1,776,391.9
1088,NaN,MA2 ÅLAND,Total,8.7,119,87.4
1089,NaN,NaN,"ENTERPRISE SECTOR, TOTAL",7.6,99,79.7
1090,NaN,NaN,"GOVERNMENT SECTOR + PRIVATE NON-PROFIT SECTOR,...",.,.,.


In [118]:
row_split = 84
year_batch = {}

for i in range(13):
    chunk = df.iloc[i*row_split:(i+1)*row_split]
    year = chunk['Unnamed: 0'].iloc[0]
    year_batch[year] = chunk.drop('Unnamed: 0', axis=1).ffill()

In [119]:
year_batch[2014]

,Unnamed: 1,Unnamed: 2,R&D expenditure (Mill. EUR),R&D personnel (HC),R&D in full-time equivalents (FTE)
252,WHOLE COUNTRY,Total,6512.1,77306,52130.2
253,WHOLE COUNTRY,"ENTERPRISE SECTOR, TOTAL",4409.5,40217,29633.9
254,WHOLE COUNTRY,"GOVERNMENT SECTOR + PRIVATE NON-PROFIT SECTOR,...",613.1,8280,6462.4
255,WHOLE COUNTRY,"HIGHER EDUCATION SECTOR, TOTAL",1489.5,28809,16033.9
256,MA1 MAINLAND FINLAND,Total,6508.9,77227,52094.3
...,...,...,...,...,...
331,MK19 Lapland,"HIGHER EDUCATION SECTOR, TOTAL",28.1,603,289
332,MA2 ÅLAND,Total,3.2,80,35.9
333,MA2 ÅLAND,"ENTERPRISE SECTOR, TOTAL",2.7,63,28.5
334,MA2 ÅLAND,"GOVERNMENT SECTOR + PRIVATE NON-PROFIT SECTOR,...",-,-,-


In [120]:
for i in year_batch:
    df_pivoted = year_batch[i].pivot(index=['Unnamed: 1'], 
                        columns='Unnamed: 2', 
                        values=['R&D expenditure (Mill. EUR)', 'R&D personnel (HC)', 'R&D in full-time equivalents (FTE)'])

    # Flatten column names
    df_pivoted.columns = [f"{col[0]} for {col[1]}" for col in df_pivoted.columns]
    year = [str(int(i))] * 21
    muni = ['Total']*21
    df_pivoted['year'] = year
    df_pivoted['municipality'] = muni
    df_pivoted = df_pivoted.reset_index()
    df_pivoted = df_pivoted.rename(columns={'Unnamed: 1': 'region'})
    df_pivoted = df_pivoted[df_pivoted['region'].str.contains("MK")]
    df_pivoted['region'] = df_pivoted['region'].str.replace('MK', '', regex=False).str.split(' ').str[0]
    year_batch[i] = df_pivoted


In [121]:
final = pd.concat(year_batch.values(), ignore_index=True)

In [122]:
cols = list(final.columns.values)
cols.insert(1, cols.pop(cols.index('municipality')))
cols.insert(2, cols.pop(cols.index('year')))
final = final[cols]

In [123]:
final

,region,municipality,year,"R&D expenditure (Mill. EUR) for ENTERPRISE SECTOR, TOTAL","R&D expenditure (Mill. EUR) for GOVERNMENT SECTOR + PRIVATE NON-PROFIT SECTOR, TOTAL","R&D expenditure (Mill. EUR) for HIGHER EDUCATION SECTOR, TOTAL",R&D expenditure (Mill. EUR) for Total,"R&D personnel (HC) for ENTERPRISE SECTOR, TOTAL","R&D personnel (HC) for GOVERNMENT SECTOR + PRIVATE NON-PROFIT SECTOR, TOTAL","R&D personnel (HC) for HIGHER EDUCATION SECTOR, TOTAL",R&D personnel (HC) for Total,"R&D in full-time equivalents (FTE) for ENTERPRISE SECTOR, TOTAL","R&D in full-time equivalents (FTE) for GOVERNMENT SECTOR + PRIVATE NON-PROFIT SECTOR, TOTAL","R&D in full-time equivalents (FTE) for HIGHER EDUCATION SECTOR, TOTAL",R&D in full-time equivalents (FTE) for Total
0,01,Total,2011,2111.5,428.1,562.8,3102.4,18441,6017,11153,35611,13862.5,4470.2,5918,24250.7
1,02,Total,2011,561.4,18.9,166.5,746.8,3481,308,3829,7618,2712.7,221.1,1962.5,4896.3
2,04,Total,2011,91.9,2.9,14.1,109,920,32,318,1270,604.6,29,187.4,821
3,05,Total,2011,39.7,34.8,10,84.5,620,521,321,1462,351.2,474.6,130.4,956.2
4,06,Total,2011,873.6,45.7,154.2,1073.5,5796,582,3657,10035,4711.5,482.4,1873.2,7067.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,15,Total,2023,284.5,0.4,44.9,329.8,1616,8,878,2502,1396.5,3.5,464.5,1864.5
230,16,Total,2023,25.2,2,10.8,37.9,168,33,210,412,115.8,14,128.5,258.3
231,17,Total,2023,710.1,46.7,194.8,951.6,5607,439,3902,9948,4527.5,345.5,2088.8,6961.7
232,18,Total,2023,15.9,2.6,10.6,29,158,37,229,424,92.2,18.8,126.5,237.5
